# Notebook 2: Feature Engineering

This notebook demonstrates the feature engineering pipeline for nuclear
cross-section prediction. We examine:

1. Nuclear structure features derived from AME2020 and NuBase2020
2. Missing value analysis and KNN imputation
3. Feature distribution analysis
4. Feature importance from a simple baseline model
5. Physics-motivated interaction features

The feature vector captures nuclear structure information accessible for
all isotopes, including those with no measured cross-sections.

In [ ]:
import sys
sys.path.insert(0, '../src')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

from data.nuclear_properties import NuclearPropertiesLoader, MAGIC_PROTON, MAGIC_NEUTRON
from features.feature_engineering import FeatureEngineer, FEATURE_COLUMNS, add_physics_interactions

plt.rcParams.update({'font.family': 'serif', 'figure.dpi': 150})
print('Setup complete.')
print(f'Number of features: {len(FEATURE_COLUMNS)}')
print('Features:', FEATURE_COLUMNS)

## 1. Load nuclear properties

The AME2020 mass table provides binding energies and separation energies.
NuBase2020 provides spin, parity, and half-life information.
FRDM provides nuclear deformation parameters.

In [ ]:
loader = NuclearPropertiesLoader(
    ame_path='../data/external/ame2020_mass.txt',
    nubase_path='../data/external/nubase2020.txt',
    frdm_path='../data/external/frdm2012_deformation.dat',
)

ame_path = Path('../data/external/ame2020_mass.txt')
if ame_path.exists():
    props = loader.load_all()
    print(f'Loaded properties for {len(props)} isotopes')
    print(f'Available columns: {list(props.columns)}')
    display(props.head(10))
else:
    print('Nuclear properties files not found. Generating synthetic data...')
    # Synthetic data for demonstration
    rng = np.random.default_rng(0)
    Z_range = np.arange(1, 100)
    records = []
    for Z in Z_range:
        for dN in range(-5, 30):
            N = Z + dN
            if N < 0:
                continue
            A = Z + N
            be = 8.5 - 0.01*(A - 60)**2/A
            records.append({'Z': Z, 'N': N, 'A': A,
                            'binding_energy_per_A_MeV': be + rng.normal(0, 0.1),
                            'Sn_MeV': 8.0 + rng.normal(0, 0.5),
                            'S2n_MeV': 14.0 + rng.normal(0, 1.0),
                            'Sp_MeV': 8.0 + rng.normal(0, 0.5),
                            'S2p_MeV': 14.0 + rng.normal(0, 1.0),
                            })
    props = pd.DataFrame(records)
    from data.nuclear_properties import NuclearPropertiesLoader
    tmp = NuclearPropertiesLoader()
    props = tmp._compute_derived_features(props)
    print(f'Generated synthetic properties for {len(props)} isotopes')

## 2. Feature distributions

In [ ]:
# Plot distributions of key nuclear structure features
plot_features = [
    'binding_energy_per_A_MeV', 'Sn_MeV', 'S2n_MeV',
    'isospin_asym', 'dist_to_magic_Z', 'dist_to_magic_N',
]
available_features = [f for f in plot_features if f in props.columns]

n_cols = 3
n_rows = (len(available_features) + n_cols - 1) // n_cols
fig, axes = plt.subplots(n_rows, n_cols, figsize=(12, 4 * n_rows))

for ax, feat in zip(axes.flat, available_features):
    data = props[feat].dropna()
    ax.hist(data, bins=50, color='steelblue', edgecolor='none', alpha=0.8)
    ax.set_xlabel(feat)
    ax.set_ylabel('Count')
    ax.set_title(f'{feat}\n(n={len(data)}, mean={data.mean():.2f})')
    ax.spines[['top', 'right']].set_visible(False)

# Hide unused axes
for ax in list(axes.flat)[len(available_features):]:
    ax.set_visible(False)

plt.suptitle('Nuclear structure feature distributions (AME2020)', y=1.01)
plt.tight_layout()
plt.savefig('../results/figures/feature_distributions.pdf', dpi=300, bbox_inches='tight')
plt.show()

## 3. Missing value analysis

Not all nuclear properties are known for all isotopes. We examine the
pattern of missing values and demonstrate KNN imputation.

In [ ]:
# Missing value rates per feature
missing_rates = props[FEATURE_COLUMNS].isnull().mean().sort_values(ascending=False)

fig, ax = plt.subplots(figsize=(8, 5))
missing_rates[missing_rates > 0].plot.barh(ax=ax, color='salmon', edgecolor='none')
ax.set_xlabel('Fraction of isotopes with missing value')
ax.set_title('Missing value rates in nuclear structure feature set')
ax.axvline(0.1, color='red', ls='--', alpha=0.7, label='10% threshold')
ax.legend()
ax.spines[['top', 'right']].set_visible(False)
plt.tight_layout()
plt.savefig('../results/figures/missing_value_rates.pdf', dpi=300, bbox_inches='tight')
plt.show()

print('\nFeatures with missing values:')
print(missing_rates[missing_rates > 0].to_string())

## 4. Nuclear chart heatmaps for key features

Visualizing features on the (N, Z) chart reveals the shell structure
and isotopic trends that ML models should learn.

In [ ]:
if 'binding_energy_per_A_MeV' in props.columns:
    be_data = props.dropna(subset=['binding_energy_per_A_MeV'])
    
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    # Binding energy per nucleon
    sc1 = axes[0].scatter(
        be_data['N'], be_data['Z'],
        c=be_data['binding_energy_per_A_MeV'],
        cmap='RdYlGn', s=5, marker='s',
        vmin=0, vmax=9
    )
    cbar1 = plt.colorbar(sc1, ax=axes[0], fraction=0.046, pad=0.04)
    cbar1.set_label('B/A (MeV)')
    for m in [2, 8, 20, 28, 50, 82, 126]:
        axes[0].axvline(m, color='gray', lw=0.4, alpha=0.7, ls=':')
        axes[0].axhline(m, color='gray', lw=0.4, alpha=0.7, ls=':')
    axes[0].set_xlabel('Neutron number N')
    axes[0].set_ylabel('Proton number Z')
    axes[0].set_title('Binding energy per nucleon B/A')
    
    # Neutron separation energy
    if 'Sn_MeV' in be_data.columns:
        sn_data = be_data.dropna(subset=['Sn_MeV'])
        sc2 = axes[1].scatter(
            sn_data['N'], sn_data['Z'],
            c=sn_data['Sn_MeV'],
            cmap='viridis', s=5, marker='s',
            vmin=0, vmax=20
        )
        cbar2 = plt.colorbar(sc2, ax=axes[1], fraction=0.046, pad=0.04)
        cbar2.set_label('S_n (MeV)')
        for m in [2, 8, 20, 28, 50, 82, 126]:
            axes[1].axvline(m, color='gray', lw=0.4, alpha=0.7, ls=':')
            axes[1].axhline(m, color='gray', lw=0.4, alpha=0.7, ls=':')
        axes[1].set_xlabel('Neutron number N')
        axes[1].set_ylabel('Proton number Z')
        axes[1].set_title('One-neutron separation energy S_n')
    
    plt.tight_layout()
    plt.savefig('../results/figures/nuclear_chart_features.pdf', dpi=300, bbox_inches='tight')
    plt.show()
else:
    print('Binding energy data not available for chart plot.')

## 5. Build and inspect the full feature matrix

In [ ]:
# Load a processed dataset if available
dataset_path = Path('../data/processed/n_gamma_dataset.h5')
if dataset_path.exists():
    from data.dataset_builder import DatasetBuilder
    splits = DatasetBuilder.load_from_hdf5(str(dataset_path))
    X_train, y_train = splits['train']
    print(f'Loaded training dataset: X shape = {X_train.shape}, y shape = {y_train.shape}')
    print(f'Target (log10 sigma) range: [{y_train.min():.2f}, {y_train.max():.2f}]')
    
    # Feature correlation with target
    corr_with_target = []
    for j, feat in enumerate(FEATURE_COLUMNS):
        x_col = X_train[:, j]
        valid = ~(np.isnan(x_col) | np.isnan(y_train))
        if valid.sum() > 30:
            from scipy.stats import spearmanr
            r, p = spearmanr(x_col[valid], y_train[valid])
            corr_with_target.append({'feature': feat, 'spearman_r': r, 'p_value': p})
    
    if corr_with_target:
        corr_df = pd.DataFrame(corr_with_target).sort_values('spearman_r', key=abs, ascending=False)
        print('\nSpearman correlation with log10(cross-section):')
        display(corr_df)
else:
    print(f'Dataset not found at {dataset_path}.')
    print('Run: python scripts/build_dataset.py --reaction n-gamma')